# Rustress

Model training

In [1]:
!unzip stressdb.zip
!ls

Archive:  stressdb.zip
  inflating: stressdb.json           
README.md	  ruwiktionary.py  stressdb.zip
requirements.txt  stressdb.json    Untitled.ipynb


In [2]:
!du -h stressdb.json

111M	stressdb.json


In [1]:
import pymorphy2 as py
morph = py.MorphAnalyzer()
stress_char = "ре́с"[2]

In [2]:
import json
stressdb = json.loads(open('stressdb.json').read())

In [3]:
print(list(stressdb.items())[:10])

[('солнышек', 'со́лнышек'), ('солнышка', 'со́лнышка'), ('солнышках', 'со́лнышках'), ('солнышки', 'со́лнышки'), ('солнышками', 'со́лнышками'), ('солнышку', 'со́лнышку'), ('солнышкам', 'со́лнышкам'), ('солнышке', 'со́лнышке'), ('солнышком', 'со́лнышком'), ('солнышко', 'со́лнышко')]


In [47]:
MAX_VOW = 0
MAX_LENGTH = 0
vow = 'уеыаоэяиюё'
chars = set()
for el in stressdb:
    chars.update(el)
    curr = 0
    for c in el:
        curr += c in vow
    MAX_VOW = max(curr, MAX_VOW)
    MAX_LENGTH = max(MAX_LENGTH, len(el))
print(MAX_LENGTH)
print(chars)

26
{'ш', 'А', 'в', 'й', 'п', 'ь', 'С', 'a', 'н', 'з', 'ч', 'х', 'Я', 'К', 'ъ', 'ѝ', 'Б', 'П', 'б', 'ф', 'р', 'Е', 'Э', 'И', 'г', 'З', 'м', 'я', 'э', 'Г', 'Р', 'и', 'ы', 'д', 'Ф', 'ї', 'с', 'ж', 'о', 'л', 'і', 'Н', 'М', 'ю', 'e', 'а', 'Д', 'В', 'т', 'щ', 'у', 'ц', 'c', 'ё', 'ѐ', 'к', 'е'}


In [48]:
good = '@ёйцукенгшщзхфывапролджэячсмитьъбю'
c2id = dict()
id2c = dict()
for i in range(len(good)):
    id2c[i] = good[i]
    c2id[good[i]] = i

In [6]:
import numpy as np
#import pandas as pd

In [84]:
feat = ['aspect', 'case', 'POS', 'mood', 'gender', 'number', 'person', 'tense', 'voice', 'transitivity']

def prepare_data(key, val):
    key = key.replace('ѝ', 'й')[::-1]
    val = val.replace('ѝ', 'й')[::-1]
    word = morph.parse(key)[0]
    row = [0] * (MAX_LENGTH + len(feat))
    for i in range(len(feat)):
        #print('word.tag.' + feat[i])
        #print(word.tag.aspect)
        try:
            row[i] = eval('word.tag.' + feat[i])
        except:
            pass
    for i in range(len(key)):
        row[len(feat) + i] = c2id[key[i]] / len(c2id)
    ind = 0
    for i in range(len(val)):
        if val[i] == stress_char:
            break
        if val[i] in vow:
            ind += 1
    res = np.zeros(MAX_VOW)
    res[ind] = 1
    return (row, res)    

In [85]:
import random as rd
data = list(stressdb.items())
rd.shuffle(data)
X = []
Y = []
cnt1 = 0
cnt2 = 0
cnt3 = 0
s1 = []
s2 = []
for key, val in data[:]:
    key = key.lower()
    val = val.lower()
    if 'і' in key:
        cnt1 += 1
        s1.append([key, val])
        continue
    bad = {'ѐ', 'ѐ', 'a', 'c', 'ї'}
    ok = True
    for el in bad:
        if el in val + key:
            ok = False
            s2.append([el, val, key])
            cnt2 += 1
            break
    for el in val + key:
        if ord(el) < 255:
            ok = False
    if not ok:
        continue
    if key == val:
        cnt3 += 1
        continue
    try:
        (x, y) = prepare_data(key, val)
        X.append(x)
        Y.append(y)
    except Exception as exp:
        print(exp.args, exp)
        print(key, val)
        print(len(key), len(val))
        for el in bad:
            print(el, el in val + key)
        break

In [86]:
prepare_data('вкомпоновываться',  'вкомпоно́вываться')

([None,
  'gent',
  'NOUN',
  None,
  'femn',
  'plur',
  None,
  None,
  None,
  None,
  0.7058823529411765,
  0.7647058823529411,
  0.8823529411764706,
  0.8529411764705882,
  0.47058823529411764,
  0.4411764705882353,
  0.4117647058823529,
  0.4411764705882353,
  0.5588235294117647,
  0.20588235294117646,
  0.5588235294117647,
  0.5,
  0.7941176470588235,
  0.5588235294117647,
  0.14705882352941177,
  0.4411764705882353,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [87]:
print(len(X))

847706


In [88]:
print(cnt1, cnt2, cnt3)

18 2635 0


In [89]:
X_cp = X

In [101]:
cnts = [dict() for i in range(len(feat))]
back = [dict() for i in range(len(feat))]
for col in range(len(feat)):
    for i in range(len(X)):
        sz = len(cnts[col])
        if X[i][col] not in cnts[col]:
            cnts[col][X[i][col]] = sz + 1
            back[col][sz + 1] = X[i][col]
        X[i][col] = cnts[col][X[i][col]]

In [102]:
for col in range(len(feat)):
    for i in range(len(X)):
        sz = len(cnts[col])
        X[i][col] /= sz

In [103]:
print(X[:10])
print(Y[:10])

[[0.33333333 0.1        0.07142857 0.33333333 0.25       0.33333333
  0.25       0.25       0.33333333 0.33333333 0.82352941 0.47058824
  0.52941176 0.11764706 0.79411765 0.47058824 0.76470588 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.2        0.07142857 0.33333333 0.5        0.66666667
  0.25       0.25       0.33333333 0.33333333 0.05882353 0.17647059
  0.82352941 0.08823529 0.47058824 0.32352941 0.82352941 0.44117647
  0.82352941 0.85294118 0.14705882 0.17647059 0.58823529 0.58823529
  0.55882353 0.14705882 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.1        0.07142857 0.33333333 0.5        0.33333333
  0.25       0.25       0.33333333 0.33333333 0.17647059 0.41176471
  0.20588235 0.88235294 0.58823529 0.17647059 0.85294118 0.82352941
  0.852941

In [104]:
len(X[0])

36

In [105]:
from sklearn.model_selection import train_test_split
X = np.array(X)
Y = np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [106]:
X_train.shape

(593394, 36)

In [107]:
Y_train.shape

(593394, 12)

In [108]:
ans = [0] 

In [109]:
import keras

In [110]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_dim=36, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(MAX_VOW, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [111]:
model.fit(X_train, Y_train, epochs=30, batch_size=60)

Epoch 1/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.2180 - acc: 0.4502
Epoch 2/30
593394/593394 [==============================] - 12s 20us/step - loss: 1.1575 - acc: 0.4645
Epoch 3/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.1182 - acc: 0.4934
Epoch 4/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.0773 - acc: 0.5243
Epoch 5/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.0506 - acc: 0.5434
Epoch 6/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.0292 - acc: 0.5576
Epoch 7/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.0144 - acc: 0.5659
Epoch 8/30
593394/593394 [==============================] - 11s 19us/step - loss: 1.0015 - acc: 0.5723
Epoch 9/30
593394/593394 [==============================] - 11s 19us/step - loss: 0.9881 - acc: 0.5798
Epoch 10/30
593394/593394 [==============================] - 11s 19us/ste

KeyboardInterrupt: 

In [112]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

254312/254312 [==============================] - 4s 17us/step

acc: 62.81%


In [113]:
model.save('model.h5')
with open('c2id.json', 'w', encoding='utf-8') as file:
    print(json.dumps(c2id), file=file)
with open('back.json', 'w') as file:
    print(json.dumps(cnts), file=file)

In [114]:
!du -h model.h5

56K	model.h5
